# Slicer

This notebook was used as a pre-processing step for getting slices from MRI volumnes. The first approach consisted of getting slices on the fly (while the pipeline is running), so that we can decide in the experiment which slices we want to consider), but the first attempts showed that this step is quite CPU-consuming, mainly due to a poor or inexistent optimization. So we use a slicer and so the final dataset consists of pre-processed slices rather than whole volumes.

One of the hypothesis of this project is that perhaps working with skull-stripped MRIs could lead to better models. But this hypothesis can of course be wrong, so we need to test all the possibilities. In other words, we need the slicer to get both full-skull images and the skull-stripped ones.

Another important aspect has to do with which slices are actually relevant for the task, since many of them have no information. We are conservative at this point, since discarding so many slices at this point could be an impairment later in the project. The approach that was taken consists of including the majority of the slices but incorporating the amount of relevant info as a part of the file name. Therefore, the final decision of what slices to discard in the training stage can be taken on the fly.

This notebook needs:

- The original IXI-T1 dataset with all the NIFTI files
- The filtered deepbrain masks we obtained with the DeepBrain segmentation tool (executed in a local environment) and filtered with sklearn dbscan algorithm

This notebook generates:

- A directory structure with all the slices generated as PNG files, divided in two subfolders:
    - full => raw slices without being filtered, i.e. the full skull
    - skull-stripped => slices obtained from NIFTI volumes and then filtered by applying the corresponding DeepBrain mask

In [ ]:
# Libraries
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import glob
import nibabel as nib
from sklearn import cluster
from datetime import datetime
from PIL import Image


In [ ]:
def print_time():
    now = datetime.now().time()
    print("now =", now)       

class sliceAcquirer:    
    def __init__(self):
        self.use_masks = True
        self.tissue_amount_thresold = 50
        self.slices = []
        self.dic_idx = {}

    def get_mri_path(self):
        return '../input/ixi-t1-dataset'
    
    def get_mask_path(self):
        return '../input/ixifilteredmasks/IXI-filtered-masks'

    def mask_file_from_nii_file(self, nii_name):   
        path = self.get_mask_path()
        nii_name = str.replace(nii_name, '.nii', '')    
        mask_files = glob.glob(f'{path}/{nii_name}*.nii.mask.npy')
        return mask_files[0]

    def get_mask_from_nii_file(self, file):
        mask_file_name = self.mask_file_from_nii_file(os.path.basename(file))
        mask = np.load(mask_file_name)   
        return mask

    def get_mri_slices(self, file):
        nifti = nib.load(file).get_fdata()
       
        self.dic_idx[file] = {}
        self.dic_idx[file]['start'] = len(self.slices)
        
        volume = nifti
        if self.use_masks:
            # Apply the mask to the volume
            mask = self.get_mask_from_nii_file(file)    
            skull_stripped_volume = np.multiply(volume, mask)   

        x = np.shape(volume)[0]
        y = np.shape(volume)[1]
        z = np.shape(volume)[2]
                
        for i in range(0, z):
            slice = np.squeeze(volume[:, :, i:i+1])
            ss_slice = np.squeeze(skull_stripped_volume[:, :, i:i+1])
            unique, counts = np.unique(ss_slice, return_counts=True)
            d = dict(zip(unique, counts))
            tissue_amount = x*y - d[0.0]            
            if (tissue_amount >= self.tissue_amount_thresold):
                self.saveAsPNG(ss_slice, f"{file}.slice-{i}.ta{tissue_amount}.png", 'skull-stripped')
                self.saveAsPNG(slice, f"{file}.slice-{i}.ta{tissue_amount}.png", 'full')
                
        self.dic_idx[file]['end'] = len(self.slices) - 1
        
    def saveAsPNG(self, img_arr, filename, slice_type=''):        
        rescaled = (255.0 / img_arr.max() * (img_arr - img_arr.min())).astype(np.uint8)
        im = Image.fromarray(rescaled).convert('L')
        im.save("./images/" + slice_type + '/' + os.path.basename(filename))
    
    def get_all_slices(self):
        path = self.get_mri_path()
        nifti_files = glob.glob(f'{path}/*.nii')
        i = 0
        for nifti_file in nifti_files:
            self.get_mri_slices(nifti_file)
            i += 1
            print(i)

In [ ]:
# Instantite the slicer and run the process
print_time()
slicer = sliceAcquirer()
slicer.get_all_slices()
print_time()

# The resulting outpur is in fact the final dataset with which we will feed our models in later stages of this project
!cd ..
!zip -q ../../kaggle/working/images/skull-stripped/download.zip ../../kaggle/working/images/skull-stripped/*
!rm -f ./images/full/*.png
!rm -f ./images/skull-stripped/*.png